In [1]:
import csv
import numpy as np
import math
import random
from geneticalgorithm2 import geneticalgorithm2 as ga 
from scipy.spatial.distance import cdist
import pandas as pd
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)   

In [2]:
interactions = []
#import all the interaction matricies created beforehand and add to array (flattened versions of the matrices)
for i in range(50):
    for j in range(50):
        mat = np.loadtxt(open("interactionMats/"+str(i)+"_"+str(j)+".csv", "rb"), delimiter=",", skiprows=0)
        arr = np.array(mat).flatten()

        interactions.append(arr)

In [3]:
def fitness_func(solution):
    #add all the interaction values
    wake = 0
    ones = []

    for i in range(2500):
        if solution[i] == 1:
            ones.append(i)
    if len(ones) > 50:
        return 1000000

    for i in ones:
        tot = 0
        for j in ones:
            tot += interactions[i][j]
        wake += tot
    return wake
def make_arr(ones: np.ndarray):
    arr = [0] * 2500
    for i in ones:
        arr[i] = 1
    return np.array(arr)
def make_random(number, turbine):
    pop = []
    for i in range(number):
        turbines = np.random.choice(np.arange(0, 2500), turbine, replace = False)
        binary =  make_arr(turbines)
        wake = fitness_func(binary)

        pop.append([binary,wake])
    return np.array(pop)
def sort_tot(pop):
    #sort by the wake values
    return pop[np.argsort(pop[:,1])]

In [4]:
def inversion_mutation(bitstring,prob):
    if np.random.rand() < prob:
        s = int(np.random.rand() * len(bitstring) )
        e = len(bitstring) - int((np.random.rand() * min(20,(len(bitstring) - s)))) - 1

        new_string =np.flip(np.copy(bitstring[s:e+1]))

        for i in range(s,e+1):

            bitstring[i] = new_string[i-s]
    return bitstring
def scramble_mutation(bitstring,prob):
    if np.random.rand() < prob:
        s = int(np.random.rand() * len(bitstring) )
        e = len(bitstring) - int((np.random.rand() * min(20,(len(bitstring) - s)))) - 1

        new_string =np.copy(bitstring[s:e+1])
        np.random.shuffle(new_string)
        for i in range(s,e+1):
            bitstring[i] = new_string[i-s]
    return bitstring
def swap_mutation(bitstring,prob):
    
    if np.random.rand() < prob:
        lens = len(bitstring)
        i = int(np.random.rand() * lens)

        # flip the bit and one with one on opposite end
        curr = np.copy(bitstring[i])
        prev = np.copy(bitstring[lens - i - 1])
        bitstring[i] = prev
        bitstring[lens - i - 1] = curr

    return bitstring
    
def rand_mutation(bitstring, prob):
   
    if np.random.rand() < prob:
        lens = len(bitstring)
        i = int(np.random.rand() * lens)

        # flip the bit and one with opposite value (to keep same number of turbines)
        prev = np.copy(bitstring[i])
        opp = np.where(bitstring == (1-prev))[0]
        bitstring[i] = 1 - np.copy(prev)

        bitstring[  opp[int(np.random.rand()*len(opp))]  ] = np.copy(prev)
    return bitstring

In [5]:
def uniform_crossover(x: np.ndarray, y: np.ndarray):
    #randomly selects 1's from either sides
    x1 = np.where(x == 1)[0]
    y1 = np.where(y == 1)[0]

    ofs1 = []
    ofs2 = []
    
    all = list(set(x1) | set(y1))

    ofs1 = random.sample(all,50)
    ofs2 = random.sample(all,50)


    ofs1 = make_arr(ofs1)
    ofs2 = make_arr(ofs2)    
    return [ofs1, ofs2]
def segment_crossover(x: np.ndarray, y: np.ndarray):
    #randomly switch between positions of 1's in order
    x1 = np.where(x == 1)[0]
    y1 = np.where(y == 1)[0]

    ofs1 = []
    ofs2 = []
    
    common = list(set(x1).intersection(y1))
    x_unique = list(set(x1) - set(common))
    y_unique = list(set(y1) - set(common))
    sizes = len(x_unique)
    if sizes == 0:
        #the two are the same (no point in performing crossover)
        return x,y
    p = np.random.random(sizes) < 0.5
    for i, val in enumerate(p):
        if val:
            x_unique[i], y_unique[i] = y_unique[i], x_unique[i]

    ofs1 = make_arr(common+x_unique)
    ofs2 = make_arr(common+y_unique)    
    return [ofs1, ofs2]
def one_point_crossover(x: np.ndarray, y: np.ndarray):
    #randomly select a point (and select one positions from other list after this point)
    x1 = np.where(x == 1)[0]
    y1 = np.where(y == 1)[0]

    ofs1 = []
    ofs2 = []
    
    common = list(set(x1).intersection(y1))
    x_unique = list(set(x1) - set(common))
    y_unique = list(set(y1) - set(common))
    sizes = len(x_unique)
    if sizes == 0:
        #the two are the same (no point in performing crossover)
        return x,y
    pt=np.random.randint(0, sizes)
    
    ofs1 = x_unique
    ofs2 = y_unique
    ofs1[:pt] = y_unique[:pt]
    ofs2[:pt] = x_unique[:pt]
    ofs1 = make_arr(common+ofs1)
    ofs2 = make_arr(common+ofs2)    
    return [ofs1, ofs2]
def two_point_crossover(x: np.ndarray, y: np.ndarray):
    #randomly select 2 points to create segment (and swap)
    x1 = np.where(x == 1)[0]
    y1 = np.where(y == 1)[0]

    ofs1 = []
    ofs2 = []
    
    common = list(set(x1).intersection(y1))
    x_unique = list(set(x1) - set(common))
    y_unique = list(set(y1) - set(common))
    sizes = len(x_unique)
    if sizes == 0:
        #the two are the same (no point in performing crossover)
        return x,y
    pt1=np.random.randint(0,sizes)
    pt2=np.random.randint(pt1, sizes)
    
    ofs1 = x_unique
    ofs2 = y_unique
    ofs1[pt1:pt2] = y_unique[pt1:pt2]
    ofs2[pt1:pt2] = x_unique[pt1:pt2]
    ofs1 = make_arr(common+ofs1)
    ofs2 = make_arr(common+ofs2)    
    return [ofs1, ofs2]

In [6]:
def roulette_selection(pop,proportion):
    pairs = [] #indexes of parents 
    #select pairs based on probs with replacements
    scores = 1/pop[:,1]
    tot = np.sum(scores)

    probs = (scores/tot).astype('float64')
    parents = np.random.choice(a=np.arange(0,scores.size),size=int(scores.size*proportion),p=probs,replace=True)
    
    for i in range(int(parents.size/2)):
        #print("Parents:",parents[i*2]," ",parents[(i*2)+1])
        if parents[i*2] != parents[(i*2)+1]:
            pairs.append([pop[parents[i*2]][0],pop[parents[(i*2)+1]][0]])
        else:
            new = np.random.randint(0,high=scores.size)
            while new != parents[i*2]:
                new = np.random.randint(0,high=scores.size)
            pairs.append([pop[parents[i*2]][0],pop[new][0]])
            
    return pairs



def rank_probs(pop,N):
    SP = 1.5 #probability of choosing fitter individual / probability of choosing avg individual between [1,2]
    scores = pop[:,1]
    ranks = np.argsort(scores)
    new_scores = N - ((2 - SP) + (2*(SP - 1)*(ranks/(N-1))) )
    probs = new_scores / np.sum(new_scores)
    return probs
def rank_selection(pop,proportion):
    #select pairs based on probabilities defined by a secondary function above
    pairs = [] #indexes of parents 
    scores = pop[:,1]
    N = int(scores.size*proportion)

    probs = rank_probs(pop,N)
    parents = np.random.choice(a=np.arange(0,scores.size),size=N,p=probs,replace=True)

    for i in range(int(N/2)):
        if parents[i*2] != parents[(i*2)+1]:
            pairs.append([pop[parents[i*2]][0],pop[parents[(i*2)+1]][0]])
        else:
            new = np.random.randint(0,high=scores.size)
            while new != parents[i*2]:
                new = np.random.randint(0,high=scores.size)
            pairs.append([pop[parents[i*2]][0],pop[new][0]])
    return pairs



def tournament_selection(pop,proportion):
    #ramdomly select 4 parents and pick 2 in each block according to thier fitness
    pairs = [] #indexes of parents 
    scores = pop[:,1]
    parents = np.random.choice(a=np.arange(0,scores.size),size=(int(scores.size*proportion)*2),replace=True)

        
    for i in range(int(parents.size/4)):

        ratings1 = np.argsort(pop[parents[i*4:(i*4)+2],1])
        ratings2 = np.argsort(pop[parents[i*4+2:(i*4)+4],1])

        if ratings1[0] != ratings2[0]:
            pairs.append([pop[parents[(i*4)+ratings1[0]]][0],pop[parents[(i*4)+ratings2[0]]][0]])
        else:


            new2 = ratings2[1]
            while new2 != ratings1[0]:
                #we pick random since both selected for 2 are same as 1 (we do not want to use that picked for round 1 since it is known to be worse)
                new2 = np.random.randint(0,high=scores.size)

            pairs.append([pop[parents[(i*4)+ratings1[0]]][0],pop[new2][0]])

    return pairs

In [7]:
def ga(mutationT, crossoverT,selectionT,popN,iter):
    #globals()[mutationT]()
    pop =  make_random(popN,50)
    pop = sort_tot(pop)
    parent_prob = 0.7 #percent of parents used to make children
    child_next_gen = 0.5 #percent of children selected to be in next generation
    prob_mutation = 0.1 #probability of a child going through mutation

    children_nextN = int(child_next_gen*popN)
    parent_nextN = int((1-child_next_gen)*popN)
    iter_best_fitness = []

    prev_best = 10000
    iter_wo_improvement = 0
    while(iter_wo_improvement < iter):
        parent_pairs = globals()[selectionT](pop,parent_prob)
        children = []
        for [a,b] in parent_pairs:
            child_pair = globals()[crossoverT](a,b)
            c1_bin = globals()[mutationT](child_pair[0],prob_mutation)
            c2_bin = globals()[mutationT](child_pair[1],prob_mutation)
            
            children.append([c1_bin,fitness_func(c1_bin)])
            children.append([c2_bin,fitness_func(c2_bin)])
        #remake population with 50/50 split old and new - elitist
        

        children = np.array(children)
        children = sort_tot(children)
        next_pop = np.concatenate((children[:children_nextN,], pop[:parent_nextN,]))
        pop = sort_tot(next_pop)
        iter_best_fitness.append(pop[0,1])

        #while
        if prev_best == pop[0,1]:
            iter_wo_improvement += 1
        else:
            prev_best = pop[0,1]
            iter_wo_improvement = 0
    best = pop[0]
    return best, iter_best_fitness

In [8]:

mutations = ["rand_mutation","swap_mutation","scramble_mutation","inversion_mutation"]
crosses = ["uniform_crossover","segment_crossover","one_point_crossover","two_point_crossover"]
selects = ["tournament_selection","rank_selection","roulette_selection"]

best_results = dict.fromkeys(["Mutation","Crossover","Selection","Iterations","Round","Result"], [])

for m in mutations:
    for c in crosses:
        for s in selects:
            for i in range(5):
                best,it = ga(crossoverT=c,mutationT=m,selectionT=s, popN=500,iter=50)
                bestarr = best[0]
                bestmatt = bestarr.reshape((50,50))
                np.savetxt("gaResultsAll/"+m+"_"+c+"_"+s+"_"+"R"+str(i)+ ".csv", bestmatt.astype('int32'), delimiter = ",",fmt='% 4d')

                bestval = best[1]

                best_results['Mutation'] = best_results['Mutation'] + [m]
                best_results['Crossover'] = best_results['Crossover'] + [c]
                best_results['Selection'] = best_results['Selection'] + [s]
                best_results['Iterations'] = best_results['Iterations'] + [len(it)]
                best_results['Round'] = best_results['Round'] + [str(i)]
                best_results['Result'] = best_results['Result'] + [bestval]



GA_results = pd.DataFrame.from_dict(best_results)
GA_results.to_csv("GA_results.csv",index=False)


KeyboardInterrupt: 

In [ ]:
# import matplotlib.pyplot as plt
# best,it = ga(crossoverT="one_point_crossover",mutationT="rand_mutation",selectionT="roulette_selection",popN=500,iter=50)
# plt.plot(range(len(it)), it)
# bestarr = best[0]
# bestarr2 = bestarr.reshape((50,50))

# np.savetxt("best_test.csv", bestarr2.astype('int32'), delimiter = ",",fmt='% 4d')
#np.random.choice(a=np.arange(0,temp[:,1].size),size=int(temp[:,1].size*0.1),replace=True,p=(temp[:,1]/np.sum(temp[:,1])))